# Timeline
- 2017-10-05 - Started

# Goal
Get better data about persuasiveness by:
- asking pairwise comparisons, within-subject where possible
- asking an impersonal question


## Get the analysis data

In [24]:
import os
import json
import pandas as pd
import toolz


In [3]:
os.chdir(os.path.expanduser('/Users/kcarnold/code/suggestion/'))

In [7]:
%run -m suggestion.aggregate_analysis -- --batch persuade_0

Inserted but backspaced... 3mr4jp tail tail
Inserted but backspaced... 7qc24j mily mil
Inserted but backspaced... 7qc24j friendly friend
Inserted but backspaced... 7qc24j go g
Inserted but backspaced... wgfq8r staurants stauran
Inserted but backspaced... v7g5x5 ence ence
Inserted but backspaced... v7g5x5 f f
Inserted but backspaced... v7g5x5 sh sh
Inserted but backspaced... v7g5x5 cles cle
Inserted but backspaced... v7g5x5 fs f
________________________________________________________________________________
[Memory] Calling suggestion.analysis_util.get_log_analysis_raw...
get_log_analysis_raw(PosixPath('/Users/kcarnold/code/suggestion/logs/5p85g8.jsonl'), 1611657, git_rev='ab04f1a', analysis_files={ 'analysis': '#!/bin/bash\n'
              'cd "$(dirname ${BASH_SOURCE[0]})"\n'
              'NODE_ENV=development ./node_modules/babel-cli/bin/babel-node.js '
              '--presets env,react-app analyze.js\n',
  'analyze.js': "import {analyzeLog} from './src/Analyzer.js';\n"
          

Construct pairs within subjects.

In [27]:
pairs = []
for participant_id, trials in trial_level_data[trial_level_data.argue_pro].groupby('participant_id'):
    pairs.append(dict(
        meta=dict(participant_id=participant_id),
        texts=trials.filter('block,condition,final_text'.split(',')).to_dict('records')))
    
pairs    

[{'meta': {'participant_id': '3mr4jp'},
  'texts': [{'block': 1,
    'condition': 'sentneg',
    'final_text': "if i could give this place six stars i would. i love eating at this restaurant. they always have new specials and keep their classics as staples on the menu. they use local ingredients that are grown within a hundred mile radius of the restaurant. it's close to a lot of offices and is great for a working lunch or to take a client out for a good time. "},
   {'block': 2,
    'condition': 'sentpos',
    'final_text': 'i recently visited this restaurant for the first time for my birthday. everything about this place is amazing - the ambiance, coziness, food, service, cocktails - i could go on. it was a little pricier than most places in town, but the food is just great and the staff is always very friendly. i would definitely recommend this place for anyone looking for a good local flavor. '}]},
 {'meta': {'participant_id': '5p85g8'},
  'texts': [{'block': 0,
    'condition': 's

Generate attention-check phrases: one phrase that did occur in the pair, one that didn't.

In [50]:
import random
def rand_substring(s, num_words):
    words = s.split()
    start_idx = random.randrange(len(words) - num_words)
    return ' '.join(words[start_idx:start_idx + num_words])

num_words = 5
check_texts = []
for idx, pair in enumerate(pairs):
    while True:
        which = random.randrange(2)
        true_text = pair['texts'][which]['final_text']
        true = rand_substring(true_text, num_words)
        if true in true_text and all(true not in text['final_text'] for idx2, pair in enumerate(pairs) if idx != idx2 for text in pair['texts']):
            check_texts.append(true)
            break
        

In [54]:
assert len(check_texts) == len(pairs)
for idx, check_text in enumerate(check_texts):
    pairs[idx]['check_texts'] = dict(good=check_text, bad=check_texts[(idx + 6) % len(check_texts)])


Create batches of 6 pairs, write to CSV.

In [55]:
def batchify_to_csv(items, batch_size, out_fname):
    batches = list(toolz.partition_all(batch_size, items))
    if len(batches[-1]) != batch_size:
        print("Tacking on extra to the last batch.")
        batches[-1] = (batches[-1] + batches[0])[:batch_size]
    assert len(batches[-1]) == batch_size
    pd.DataFrame(dict(task=[json.dumps(batch) for batch in batches])).to_csv(out_fname, index=False)

batchify_to_csv(pairs, 6, '/Users/kcarnold/code/suggestion/annotation_ui/pairwise-persuasive-task.csv')

Tacking on extra to the last batch.
